In [1]:
import torch
import torch.nn.functional as F
import re
from pathlib import Path
from transformers import PreTrainedTokenizerFast
from models import NextByteTransformer

context_length = 512
d_model = 512
num_heads = 8
num_hidden_layers = 8
d_hidden = 2048
num_decoders = 2
num_epochs = 8
lr = 3e-5
batch_size = 16
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_path = Path("Tokenizers/title_to_all_tokenizer")
hf_tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
hf_tokenizer.eos_token_id = hf_tokenizer.convert_tokens_to_ids("<end>")
model = NextByteTransformer(
    vocab_size=20000,
    context_length=context_length,
    d_model=d_model,
    num_heads=num_heads,
    num_hidden_layers=num_hidden_layers,
    d_hidden=d_hidden,
    num_decoders=num_decoders
).to(device)

model.load_state_dict(torch.load("Models/all.pth", map_location=device))
model.eval()

def generate_autoregressive(model, tokenizer, input_text, max_new_tokens=100, top_k=10, context_length=512, device="cpu"):
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    input_ids = input_ids[:, -context_length:]
    generated = input_ids

    with torch.no_grad():
        for _ in range(max_new_tokens):
            if generated.size(1) > context_length:
                generated = generated[:, -context_length:]

            logits = model(generated)[:, -1, :]  # shape: [1, vocab_size]
            topk_logits, topk_indices = torch.topk(logits, k=top_k, dim=-1)
            probs = F.softmax(topk_logits, dim=-1)

            sampled_index = torch.multinomial(probs, num_samples=1)  # shape: [1, 1]
            next_token = topk_indices.gather(-1, sampled_index)  # shape: [1, 1]

            generated = torch.cat([generated, next_token], dim=1)

            if next_token.item() == tokenizer.eos_token_id:
                break

    return tokenizer.decode(generated[0], skip_special_tokens=False)

def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")

count_parameters(model)

Total parameters: 75,312,672
Trainable parameters: 75,312,672


In [3]:
input_text = "eggplant parmesan <end_title>"
output = generate_autoregressive(
    model=model,
    tokenizer=hf_tokenizer,
    input_text=input_text,
    max_new_tokens=400,
    top_k=10,
    context_length=context_length,
)
title = re.sub(r'\s+([.,!?;:])', r'\1', output[:output.index("<end_title>")])
ingredients = re.sub(r'\s+([.,!?;:])', r'\1', output[output.index("<end_title>") + len("<end_title>"):output.index("<end_ingredients>")])
directions = re.sub(r'\s+([.,!?;:])', r'\1', output[output.index("<end_ingredients>") + len("<end_ingredients>"):-len("<end>")])

print(f"Recipe: \n{title.title()}\n")
print(f"Ingredients: \n{ingredients.title()}\n")
print(f"Directions: \n{directions.title()}\n")

Recipe: 
Eggplant Parmesan 

Ingredients: 
 1 Small Eggplant ( About 3 To 4 12 Lb ), 12 Cup Grated Asiago Cheese ( About 6 Oz. ), 1 Tbsp. Extra - Virgin Olive Oil, 14 Oz. Parmesan Cheese, 14 C. Finely Chopped Fresh Parsley, Divided, 1 Tbsp. Finely Chopped Fresh Thyme, Divided Divided, 1 12 Tbsp. Grated Parmesan Cheese 

Directions: 
 Preheat Oven To 450 Degrees And Butter A 2 - Quart Baking Dish. Peel Eggplant, Leaving Stems. Place Each Eggplant Half On A Paper Towel - Lined Plate. Broil For 5 Minutes Or Until Eggplant Is Lightly Browned ( The Skin Should Be Browned ). Meanwhile, Combine Mozzarella, 12 Cup Ricotta, 2 Tbsp. Parmesan, Parsley, And 1 Cup Of The Parmesan In A Bowl, And Toss To Coat. Divide Eggplant And 14 Cup Of The Parmesan Over Eggplant To Within 12 Inch Of The Edge, And Scatter On 2 Baking Sheets. Bake Until The Eggplant Is Tender And Lightly Browned, About 30 Minutes. Transfer Eggplant To A Cutting Board And Cut Into 12 - Inch Chunks. 

